In [1]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

<p style="text-align:center;font-size:20px;color:green;font-weight:bold;font-family:'微软雅黑';"> rnn特别擅长处理序列类型的数据，因为他是一个循环的结构</p>
<img src="https://ptorch.com/uploads/9d761e90e601fe6cf4bd8dd605d28935.png">
<p style="text-intend:2em;font-size:16px;color:blue;font-family:'微软雅黑';"> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;一个序列的数据依次进入网络A，网络A循环的往后传递。
这就是RNN的基本结构类型。而最早的RNN模型，序列依次进入网络中，之前进入序列的数据会保存信息而对后面的数据产生影响，所以RNN有着记忆的特性，而同时越前面的数据进入序列的时间越早，所以对后面的数据的影响也就越弱，简而言之就是一个数据会更大程度受到其临近数据的影响。</p>

<p style="text-align:center;font-size:20px;color:green;font-weight:bold;font-family:'微软雅黑';"> LSTM(我们很有可能需要更长时间之前的信息，而这个能力传统的RNN特别弱)</p>
<img src="https://ptorch.com/uploads/c50692f81171b513054012fdc46266cd.png">
<p style="text-intend:2em;font-size:16px;color:blue;font-family:'微软雅黑';">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;这就是LSTM的模型结构，也是一个向后传递的链式模型，而现在广泛使用的RNN其实就是LSTM，序列中每个数据传入LSTM可以得到两个输出，而这两个输出和序列中下一个数据一起又作为传入LSTM的输入，然后不断地循环向后，直到序列结束。</p>

<a href="https://my.oschina.net/earnp/blog/1113893">==》参考资料</a>

In [2]:
device = torch.device('cuda:0') #使用序号为0的GPU

##  <span style="font-size:16px;color:blue;font-family:'微软雅黑';">首先需要定义好LSTM网络，需要nn.LSTM()，首先介绍一下这个函数里面的参数:</span>

- ### `input_size` 表示的是输入的数据维数 

- ### `hidden_size` 表示的是输出维数

- ### `num_layers` 表示堆叠几层的LSTM，默认是1

- ### `bias` True 或者 False，决定是否使用bias

- ### `batch_first` True 或者 False，因为nn.lstm()接受的数据输入是(序列长度，batch，输入维数)，这和我们cnn输入的方式不太一致，所以使用batch_first，我们可以将输入变成(batch，序列长度，输入维数)

- ### `dropout` 表示除了最后一层之外都引入一个dropout

- ### `bidirectional` 表示双向LSTM，也就是序列从左往右算一次，从右往左又算一次，这样就可以两倍的输出

In [3]:
sequence_length = 28
input_size = 28 #表示的是输入的数据维数
hidden_size = 128 #表示的是输出维数
num_layers = 2 #表示堆叠几层的LSTM，默认是1
num_classes = 10
batch_size = 100
num_epochs = 2
learning_rate = 0.01

In [4]:
train_dataset = torchvision.datasets.MNIST(root='./data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

In [9]:
class RNN_Model(nn.Module):
    def __init__(self,input_size,hidden_size,num_layers,num_classes):
        super(RNN_Model,self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size,num_classes)
    def forward(self,x):
        h0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers,x.size(0),self.hidden_size).to(device)
        
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])#out = out[:, -1, :]通过这种方式，out中的三个维度分别表示batch_size，序列长度和数据维度，所以中间的序列长度取-1，表示取序列中的最后一个数据，这个数据维度为128，再通过分类器，输出10个结果表示每种结果的概率。
        return out
        

In [10]:
model = RNN_Model(input_size,hidden_size,num_layers,num_classes).to(device)

In [11]:
print(model)

RNN_Model(
  (lstm): LSTM(28, 128, num_layers=2, batch_first=True)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)


In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/2], Step [100/600], Loss: 0.5269
Epoch [1/2], Step [200/600], Loss: 0.1830
Epoch [1/2], Step [300/600], Loss: 0.1193
Epoch [1/2], Step [400/600], Loss: 0.1721
Epoch [1/2], Step [500/600], Loss: 0.2878
Epoch [1/2], Step [600/600], Loss: 0.1324
Epoch [2/2], Step [100/600], Loss: 0.2253
Epoch [2/2], Step [200/600], Loss: 0.1271
Epoch [2/2], Step [300/600], Loss: 0.1875
Epoch [2/2], Step [400/600], Loss: 0.0440
Epoch [2/2], Step [500/600], Loss: 0.0246
Epoch [2/2], Step [600/600], Loss: 0.0586


In [14]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, sequence_length, input_size).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total)) 

# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 97.14 %
